# Tool for BER vs. tempreature and errors corelation 

In [146]:
import tarfile
import os
import glob
import re
import shutil
import pandas as pd

Choose:
- path to folder with logs
- name of file with lost frames
- name of file with temprature records
- pattern of logs with errors

In [147]:
path = "C:/Users/domanski/Desktop/SUD.TS0.TC9_SUD.ASIB_ABIC.ASIB1_ABIC123_ALL_10G.FSIP1.FBIP1.FBIP2.FBIP3.20190323.061613/"
berfile = "BER_PRBS_64.csv"
tempfile = "1_TC9_SUD_ASIB1_ABIC123_ALL_10G_temperatures.csv"
filesuffix = "_TESTS.TS0.TC9_SUD.BBU.SM1.SUB1."

In [149]:
#filtering rounds with frame loss
filename = path + berfile
ber_df = pd.read_csv(filename,sep=';')
ber_df = ber_df[['TestCycle', 'Timestamp', 'Frames Delta']]
ber_df['Timestamp'] = ber_df['Timestamp'].str[:8]
ber_df = ber_df[ber_df['Frames Delta']>0]
print(ber_df)

     TestCycle Timestamp  Frames Delta
0            1  06:23:09          7633
44          24  09:09:25           123
178         97  18:06:10             3
268        149  00:55:53             4
454        249  12:51:51         30691
466        261  15:46:23           239
538        297  19:22:50            42
554        305  20:10:52            58


In [150]:
#filtering temperatures
filename = path + tempfile
temp_df = pd.read_csv(filename,sep=';')
temp_df['Timestamp'] = temp_df['time'].str.split(' ', expand=True)[1]
temp_df['Timestamp'] = temp_df['Timestamp'].str.split('.', expand=True)[0]
temp_df.rename(columns={'FSIP1@board0_env_input':'temp'},inplace=True)
temp_df = temp_df[['Timestamp', 'temp']]
print(temp_df)

    Timestamp    temp
0    06:20:23  59.500
1    06:28:33  55.875
2    06:34:16  53.375
3    07:07:25  41.125
4    07:14:34  39.000
5    07:21:44  36.500
6    07:28:52  34.375
7    07:36:00  32.500
8    07:41:35  30.000
9    07:47:13  28.250
10   07:52:53  26.500
11   07:58:30  24.625
12   08:05:38  23.625
13   08:12:58  21.000
14   08:19:01  18.125
15   08:25:04  16.500
16   08:31:07  16.000
17   08:37:10  15.750
18   08:43:14  15.375
19   08:49:17  15.375
20   08:55:18  15.375
21   09:01:21  15.250
22   09:07:23  15.250
23   09:13:31  15.375
24   09:19:32  15.375
25   09:25:35  15.250
26   09:31:38  15.250
27   09:37:43  15.250
28   09:43:49  15.375
29   09:49:49  15.375
..        ...     ...
359  08:50:08  47.250
360  08:55:44  44.875
361  09:02:54  42.625
362  09:08:32  40.500
363  09:14:11  38.625
364  09:19:50  36.500
365  09:25:50  34.625
366  09:31:29  32.625
367  09:37:09  30.875
368  09:42:49  29.250
369  09:48:29  27.875
370  09:54:27  26.625
371  10:00:54  25.375
372  10:08

In [151]:
#corelate ber with tempretures
bertemp_df = pd.concat([ber_df,temp_df]).reset_index()
sort = True
bertemp_df['Timestamp'] = pd.to_datetime(bertemp_df['Timestamp'], format="%H:%M:%S")
bertemp_df = bertemp_df.sort_values(by=['Timestamp'])
shifted = bertemp_df.shift(periods=1)
bertemp_df = pd.concat([shifted["temp"],bertemp_df[['Frames Delta', 'TestCycle', 'Timestamp']]],axis=1)
bertemp_df = bertemp_df.loc[bertemp_df['Frames Delta'].notnull()]
bertemp_df = bertemp_df.sort_values(by=['TestCycle'])

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Table with tempreatures and frame loss

In [152]:
print(bertemp_df)

     temp  Frames Delta  TestCycle           Timestamp
0  59.500        7633.0        1.0 1900-01-01 06:23:09
1  40.500         123.0       24.0 1900-01-01 09:09:25
2  15.250           3.0       97.0 1900-01-01 18:06:10
3  38.375           4.0      149.0 1900-01-01 00:55:53
4  51.875       30691.0      249.0 1900-01-01 12:51:51
5  34.875         239.0      261.0 1900-01-01 15:46:23
6  33.000          42.0      297.0 1900-01-01 19:22:50
7  27.750          58.0      305.0 1900-01-01 20:10:52


In [153]:
errs = []
for index,rows in bertemp_df.iterrows():
    for file in glob.glob(path+str(int(rows["TestCycle"]))+filesuffix+'*.info'):
        err = []
        with open(file) as f: lineList = f.readlines()
        for line in lineList:
            if "ERROR" in line:
                line = line.split('ERROR')[1]
                err.append(line)
        errs.append(set(err))
common_errs = set.intersection(*errs)

Printing errors common for each round

In [154]:
list(common_errs)

["             /tmp/get_marvell_logs.sh: line 17: $'\\r': command not found\n",
 '   SSH  <--  RTNETLINK answers: File exists\n',
 '             Connection closed by foreign host.\n',
 "             /tmp/get_marvell_logs.sh: line 11: $'\\r': command not found\n",
 "   SSH  <--  mv: cannot stat 'marvell_logs_after.txt*': No such file or directory\n",
 "   SSH  <--  /tmp/get_marvell_logs.sh: line 5: $'\\r': command not found\n",
 "             /tmp/get_marvell_logs.sh: line 13: $'\\r': command not found\n",
 "             /tmp/get_marvell_logs.sh: line 9: $'\\r': command not found\n",
 "             /tmp/get_marvell_logs.sh: line 8: $'\\r': command not found\n",
 "             /tmp/get_marvell_logs.sh: line 12: $'\\r': command not found\n",
 "             /tmp/get_marvell_logs.sh: line 16: $'\\r': command not found\n",
 "             /tmp/get_marvell_logs.sh: line 18: $'\\r': command not found\n",
 "             /tmp/get_marvell_logs.sh: line 10: $'\\r': command not found\n",
 "         